In [0]:
%%capture

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import math
from tabulate import tabulate
import itertools

from IPython.display import display

nltk.download('stopwords')

In [0]:
data = pd.read_csv('https://raw.githubusercontent.com/dnrocha1/information_retrieval/master/lab02/data/results.csv')

In [0]:
#pre-process

stopwords = stopwords.words('portuguese')
documents = data['text'].apply(lambda x: x.lower())

regex = RegexpTokenizer(r'\b[A-zÀ-ú-\'\d]{3,}')
# tokens = regex.tokenize(texts)

In [0]:
M = len(documents)

def build_index(documents=documents):
  inverted_list = {}

  n_doc = 0
  for document in documents:
    token = [w for w in regex.tokenize(document) if w not in stopwords]
    counter = list(Counter(token).items())
    for elem in counter:
      key = elem[0]
      freq = elem[1]
      if key in inverted_list.keys():
        if n_doc not in inverted_list[key][0]:
          inverted_list[key].append((n_doc,freq))
      else:
        inverted_list[key] = [(n_doc,freq)]
    n_doc += 1
  
  for elem in inverted_list:
    k = len(inverted_list[elem])
    idf = math.log((M+1)/k)
    inverted_list[elem].append(idf)
  
  return inverted_list

index = build_index()

TENTAR MUDAR INTERNAMENTE NOS METODOS PRA NÃO TOKENIZAR E SIM BUSCAR A PARTIR DO INDICE

In [5]:
vocabulary = list(index.keys())

def vec_model_bin(query,document,index=index):
  q = {}
  d = {}
  
  for word in vocabulary:
    q[word] = 0
    if word in query.split():
      q[word] = 1
      
  tokens = [w for w in regex.tokenize(document) if w not in stopwords]
  for word in vocabulary:
    d[word] = 0
    if word in list(set(tokens)):
      d[word] = 1
      
  acc = 0
  for word in vocabulary:
    if q[word] > 0 and d[word] > 0:
      acc = acc + q[word]*d[word]
      
  return acc

vec_model_bin("forças armadas",documents[149])

2

In [6]:
def vec_model_tf(query,document,index=index):
  q = {}
  d = {}
  
  for word in vocabulary:
    q[word] = 0
    if word in query.split():
      q[word] = 1
  
  tokens = [w for w in regex.tokenize(document) if w not in stopwords]
  counter = Counter(tokens)
  
  for word in vocabulary:
    d[word] = 0
    if word in list(counter.keys()):
      d[word] = counter[word]
      
  acc = 0
  for word in vocabulary:
    if q[word] > 0 and d[word] > 0:
      acc = acc + q[word]*d[word]
      
  return acc

vec_model_tf("juíza federal",documents[0])

4

In [7]:
def vec_model_tf_idf(query,document,index=index):
  q = {}
  d = {}
  
  for word in vocabulary:
    q[word] = 0
    if word in query.split():
      q[word] = 1
  
  tokens = [w for w in regex.tokenize(document) if w not in stopwords]
  counter = Counter(tokens)
  
  for word in vocabulary:
    d[word] = 0
    if word in list(counter.keys()):
      d[word] = counter[word]
      
  acc = 0
  for word in vocabulary:
    idf = index[word][-1]
    if q[word] > 0 and d[word] > 0:
      acc = acc + q[word]*d[word]*idf
      
  return acc

vec_model_tf_idf("juíza federal",documents[14])

1.6502599069543555

In [8]:
vec_model_tf_idf("forças armadas", documents[149])
# vec_model_tf("forças armadas", documents[149])

33.323497612614815

In [40]:
k = 1
def bm25(query,document,index=index):
  doc_tokens = [w for w in regex.tokenize(document) if w not in stopwords]
  query_words = query.split()
  
  matched_words = list(set(doc_tokens) & set(query_words))
  
  score = 0
  
  for word in matched_words:
    cwq = query_words.count(word)
    cwd = doc_tokens.count(word)
    dfw = len(index[word][:-1])
    y = ((k+1)*cwd)/(cwd+k)
    score += cwq * y * math.log((M+1)/dfw)
    
  
  return score

bm25("forças armadas",documents[149])

7.906309290883032

In [10]:
results = []
best_k = []
for k in [x * 0.1 for x in range(0,200,5)]:
  res = bm25("forças armadas",documents[149],k)
  results.append(res)
  best_k.append(k)
  
df = pd.DataFrame({'res':results, 'k':best_k})

# df.quantile([0.25,0.5,0.75]).iloc[0,:].k
df.quantile([0.25,0.5,0.75])

,res,k
0.25,15.893550,4.875
0.50,20.831447,9.750
0.75,23.588632,14.625


In [41]:
print(bm25("forças armadas",documents[149]))
k = 4.875
print(bm25("forças armadas",documents[149]))

7.906309290883032
15.89904328901099


In [0]:
queries = ["governo federal", "golpe de estado", "forças armadas"]

In [0]:
def get_top5(query,fun):
  doc_id = 0
  scores = []
  docs = {}
  for document in documents:
    score = fun(query,document)
    docs[doc_id] = score
    doc_id += 1
    
  return sorted(docs.items(), key=lambda kv:kv[1], reverse=True)

def build_query(query):
  funs = [vec_model_bin,vec_model_tf,vec_model_tf_idf,bm25]
  table = {}
  for fun in funs:
    top = get_top5(query,fun)
    table[fun.__name__] = top
    
  return pd.DataFrame(table)

In [191]:
for query in queries:
  table = build_query(query)
  print(query)
  display(table.head(5))
  print('\n')

forças armadas


,vec_model_bin,vec_model_tf,vec_model_tf_idf,bm25
0,"(0, 2)","(149, 15)","(149, 33.323497612614815)","(149, 15.89904328901099)"
1,"(5, 2)","(24, 9)","(24, 19.860802780873335)","(24, 12.506924563304118)"
2,"(11, 2)","(165, 8)","(165, 17.95025977565531)","(165, 11.882566330363375)"
3,"(24, 2)","(207, 8)","(207, 17.95025977565531)","(207, 11.882566330363375)"
4,"(41, 2)","(0, 6)","(0, 13.462694831741484)","(0, 10.043597731616662)"




governo federal


,vec_model_bin,vec_model_tf,vec_model_tf_idf,bm25
0,"(2, 2)","(172, 19)","(172, 19.90201857230658)","(172, 8.301438203245631)"
1,"(23, 2)","(165, 14)","(247, 13.818099017189173)","(247, 7.195657384981824)"
2,"(33, 2)","(247, 13)","(165, 13.177774992178293)","(228, 6.930440834926088)"
3,"(36, 2)","(114, 12)","(114, 12.167839110234816)","(114, 6.3226627502206885)"
4,"(41, 2)","(228, 11)","(228, 12.044635157923953)","(219, 6.041405075584327)"




golpe de estado


,vec_model_bin,vec_model_tf,vec_model_tf_idf,bm25
0,"(0, 2)","(24, 14)","(24, 32.77309553455966)","(24, 11.562170753434446)"
1,"(1, 2)","(2, 8)","(2, 18.190584747536075)","(165, 9.890628428592263)"
2,"(2, 2)","(6, 8)","(165, 16.937821779040707)","(2, 9.594578388869541)"
3,"(24, 2)","(165, 8)","(164, 14.507403314536777)","(164, 9.242446114401124)"
4,"(97, 2)","(248, 8)","(207, 13.329747818528213)","(0, 8.448194546307755)"


In [82]:
# get only docs
table_docs = table.applymap(lambda x: x[0])
table_docs

,vec_model_bin,vec_model_tf,vec_model_tf_idf,bm25
0,0,24,24,24
1,1,2,2,165
2,2,6,165,2
3,24,165,164,164
4,97,248,207,0


In [262]:
pd.options.display.max_colwidth = 500
BOLD = '\033[1m'
END = '\033[0m'

print(f'A consulta realizada foi {BOLD}"{query}"{END}.\nAbaixo os resultados de cada modelo:\n')

for model in table_docs:
  docs = table_docs[model]
  df = data[['title','subtitle']].iloc[docs]
  print(f'{BOLD}{model}{END}')
  display(df)
  print('\n')

A consulta realizada foi "golpe de estado".
Abaixo os resultados de cada modelo:

vec_model_bin


,title,subtitle
0,“A sociedade foi Rubens Paiva não os facínoras que o mataram”,A decisão da juíza que proíbe as Forças Armadas de celebrarem golpe de 1964 contrapõe texto de militares ao de Ulysses Guimarães na Constituinte. Gilmar Mendes rejeita análise do tema
1,Justiça suspende decisão que proibia Forças Armadas de celebrarem golpe de 1964,Liminar havia sido concedida na sexta-feira a noite mas menos de 24 horas depois foi derrubada
2,Governo Bolsonaro prega “negacionismo histórico” sobre a ditadura,Marcos Napolitano professor da USP diz que o discurso do Governo Bolsonaro sobre o golpe de 64 está mais para negacionismo pois “tem um ponto de partida ideológico com objetivo de ocultar o passado”
24,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”,Historiador diz que as Forças Armadas nunca reconheceram os aspectos mais negativos do regime militar e que não havia ameaça imediata de implantação de um regime comunista
97,Maduro anuncia 30 dias de racionamento de energia na Venezuela,Presidente adota as medidas após os graves blecautes que deixaram o país sem água e luz no último dia 7




vec_model_tf


,title,subtitle
24,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”,Historiador diz que as Forças Armadas nunca reconheceram os aspectos mais negativos do regime militar e que não havia ameaça imediata de implantação de um regime comunista
2,Governo Bolsonaro prega “negacionismo histórico” sobre a ditadura,Marcos Napolitano professor da USP diz que o discurso do Governo Bolsonaro sobre o golpe de 64 está mais para negacionismo pois “tem um ponto de partida ideológico com objetivo de ocultar o passado”
6,“Lógica de usar torturadores da ditadura no crime foi usada nas milícias”,Aloy Jupiara coautor de 'Os porões da contravenção' fala sobre o legado de profissionalização do crime deixado pelos anos de chumbo que perdura até hoje
165,Bolsonaro manda festejar o crime,Ao determinar a comemoração do golpe militar de 1964 o antipresidente busca manter o ódio ativo e barrar qualquer possibilidade de justiça
248,As três espanholas do Estado Islâmico: “Só queremos ir embora”,EL PAÍS localiza em meio a milhares de famílias jihadistas num campo de acolhida três mulheres que viajaram com seus maridos para a Síria em 2014 e sobreviveram ao desmoronamento do califado




vec_model_tf_idf


,title,subtitle
24,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”,Historiador diz que as Forças Armadas nunca reconheceram os aspectos mais negativos do regime militar e que não havia ameaça imediata de implantação de um regime comunista
2,Governo Bolsonaro prega “negacionismo histórico” sobre a ditadura,Marcos Napolitano professor da USP diz que o discurso do Governo Bolsonaro sobre o golpe de 64 está mais para negacionismo pois “tem um ponto de partida ideológico com objetivo de ocultar o passado”
165,Bolsonaro manda festejar o crime,Ao determinar a comemoração do golpe militar de 1964 o antipresidente busca manter o ódio ativo e barrar qualquer possibilidade de justiça
164,Celebrar o golpe representa uma derrota para a democracia,A eleição de Bolsonaro e sua decisão de comemorar o golpe representam simbólica e espero provisoriamente a vitória da memória e do sentimento autoritário
207,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,"Com medida simbólica presidente determinou que Forças Armadas façam ""comemorações devidas"" da data"




bm25


,title,subtitle
24,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”,Historiador diz que as Forças Armadas nunca reconheceram os aspectos mais negativos do regime militar e que não havia ameaça imediata de implantação de um regime comunista
165,Bolsonaro manda festejar o crime,Ao determinar a comemoração do golpe militar de 1964 o antipresidente busca manter o ódio ativo e barrar qualquer possibilidade de justiça
2,Governo Bolsonaro prega “negacionismo histórico” sobre a ditadura,Marcos Napolitano professor da USP diz que o discurso do Governo Bolsonaro sobre o golpe de 64 está mais para negacionismo pois “tem um ponto de partida ideológico com objetivo de ocultar o passado”
164,Celebrar o golpe representa uma derrota para a democracia,A eleição de Bolsonaro e sua decisão de comemorar o golpe representam simbólica e espero provisoriamente a vitória da memória e do sentimento autoritário
0,“A sociedade foi Rubens Paiva não os facínoras que o mataram”,A decisão da juíza que proíbe as Forças Armadas de celebrarem golpe de 1964 contrapõe texto de militares ao de Ulysses Guimarães na Constituinte. Gilmar Mendes rejeita análise do tema


In [261]:
funs = [vec_model_bin,vec_model_tf,vec_model_tf_idf,bm25]
fun_names = [(x.__name__,y.__name__) for x,y in [tup for tup in list(itertools.combinations(funs,2))]]

print(f'Abaixo estão os resultados da consulta "{query}":\n')
for fun1,fun2 in fun_names:
  inter = len(set(table_docs[fun1]) & set(table_docs[fun2]))
  union = len(set(table_docs[fun1])) + len(set(table_docs[fun2])) - inter
  jaccard = inter/union
  print(f'O resultado entre {fun1} e {fun2} foi {jaccard:.3}.')

Abaixo estão os resultados da consulta "golpe de estado":

O resultado entre vec_model_bin e vec_model_tf foi 0.25.
O resultado entre vec_model_bin e vec_model_tf_idf foi 0.25.
O resultado entre vec_model_bin e bm25 foi 0.429.
O resultado entre vec_model_tf e vec_model_tf_idf foi 0.429.
O resultado entre vec_model_tf e bm25 foi 0.429.
O resultado entre vec_model_tf_idf e bm25 foi 0.667.
